# Skillra PDA: HSE course project

Воспроизводимый аналитический отчёт Skillra о рынке IT-вакансий (ТЗ ВШЭ, этапы 0–4) и продуктовая часть с персонами и skill-gap.

## Вводная
- Цель: собрать данных HH, очистить, обогатить признаками и провести EDA по рынку, чтобы подготовить витрину для продукта Skillra.
- Структура ноутбука следует этапам ТЗ: 0 — парсинг, 1 — предобработка, 2 — признаки, 3 — EDA (зарплаты/форматы/роли/навыки/домены/английский/образование/работодатели + корреляции), 4 — визуализации, продуктовый слой с персонами, финальные выводы и чек-лист.
- Кодовые функции живут в `src/skillra_pda`; в ноутбуке — вызовы, таблицы, графики и интерпретации.

### Команда и вклад
| --- | --- |
| Анастасия | Очистка данных, контроль качества булевых колонок |
| Максим | Фича-инжиниринг (стек, возраст вакансии, текстовые признаки) |
| Екатерина | EDA, визуализации, зарплатные срезы |
| Дмитрий | Персоны, skill-gap и продуктовые выводы |

### Настройка окружения и загрузка артефактов
Используем функции из `src.skillra_pda`, директории и пути из `config`. Если обработанные данные отсутствуют, запускаем пайплайн `scripts/run_pipeline.py`.

## Этап 0. Парсинг hh.ru и исходные данные
Парсер `parser/hse_vacancies/hh_scraper.py` собирает вакансии HH со ссылок поиска: фильтр по ИТ-запросам, странам СНГ (`DEFAULT_AREA_IDS`), только с указанной зарплатой. Встроены ротация user-agent, повторные запросы и парсинг HTML, чтобы достать грейд/роль (по ключевым словам), стек, бенефиты, soft-skills и требования к опыту/английскому/образованию. Ограничения: выборка охватывает публичные вакансии HH, могут быть смещения по регионам и валютам, а зарплатные вилки содержат только явные значения.

### 0.1 Что собирает парсер и как данные используются дальше
- Поля HH + извлечённые флаги: грейд (`junior/middle/senior/lead`), роли (`role_*`), стек (`has_python/...`), бенефиты (`benefit_*`), soft-skills.
- Параметры поиска: многострочный `DEFAULT_QUERY` по разработке/data/QA/devops/BI/архитекторам, страны СНГ (`DEFAULT_AREA_IDS`), лимит по умолчанию 10k вакансий.
- Ограничения данных: только вакансии с зарплатой, валюты приводятся к явным `RUB/USD/EUR` подсказками; возможны пропуски по описанию и грейду, которые закрываются на этапе cleaning.
- Связка с признаками: исходные поля маппятся в engineered features (`city_tier`, `work_mode`, `tech_stack_size`, `is_junior_friendly`, `salary_bucket`), которые используем в EDA и персонах.

In [ ]:

from pathlib import Path
import sys
import importlib
import pandas as pd

CWD = Path.cwd()
CANDIDATES = [CWD] + list(CWD.parents)
PROJECT_ROOT = next((p for p in CANDIDATES if (p / "src" / "skillra_pda").exists()), CWD)
SRC_DIR = PROJECT_ROOT / "src"

for path in {PROJECT_ROOT, SRC_DIR}:
    if str(path) not in sys.path:
        sys.path.insert(0, str(path))

from src.skillra_pda import config, io, cleaning, features, eda, viz, personas

config.ensure_directories()

if not config.CLEAN_DATA_FILE.exists() or not config.FEATURE_DATA_FILE.exists():
    import scripts.run_pipeline as run_pipeline
    run_pipeline.main()

raw_path = config.RAW_DATA_FILE
clean_path = config.CLEAN_DATA_FILE
feat_path = config.FEATURE_DATA_FILE

df_raw = io.load_raw(raw_path)
df_clean = pd.read_parquet(clean_path)
df_features = pd.read_parquet(feat_path)

df_features.head(2)


### 0.2 Пример сырых данных и базовые статы
Показываем head, `info()` и первичный профиль пропусков/типов, чтобы понимать качество источника до очистки.

In [ ]:

import io

raw_head = df_raw.head(5)
buffer = io.StringIO()
df_raw.info(buf=buffer)
raw_info = buffer.getvalue()

raw_overview = {
    "raw_path": str(raw_path),
    "raw_shape": df_raw.shape,
    "columns_sample": df_raw.columns.tolist()[:10],
}
raw_profile = cleaning.basic_profile(df_raw)
raw_missing_top = eda.missing_share(df_raw, top_n=10)
raw_numeric_stats = df_raw.select_dtypes("number").describe().T

raw_head, raw_info, raw_overview, raw_profile, raw_missing_top, raw_numeric_stats


### 0.3 Выводы по этапу 0
- Сырые данные: видны ключевые HH-поля (название, зарплата, описание, опыт, регион) и извлечённые парсером роли/стек/бенефиты.
- Качество: пропуски концентрируются в описаниях/бенефитах, типы колонок соответствуют словарю (`docs/02_feature_dictionary_hh.md`).
- Ограничения: сохраняем осознанность про региональный сдвиг и вакансии без зарплат (не попали в выгрузку); эти допущения учитываем в выводах.

## Этап 1. Предобработка (cleaning.py)
Повторяем шаги cleaning: парсинг дат, дедупликация, обработка пропусков, нормализация булевых колонок (`salary_gross` и все флаги is_/has_/benefit_/soft_/domain_/role_). Готовые результаты берём из `data/processed/hh_clean.parquet`. Ниже — снимки пропусков и типов.

In [ ]:

dup_raw = df_raw.duplicated().sum()
dup_clean = df_clean.duplicated().sum()
missing_salary = df_raw[[c for c in df_raw.columns if "salary" in c]].isna().mean().to_dict()

clean_missing_top = eda.missing_share(df_clean, top_n=10)
salary_gross_dtype = str(df_clean.get("salary_gross", pd.Series(dtype="boolean")).dtype)
salary_gross_stats = df_clean.get("salary_gross", pd.Series(dtype="boolean")).value_counts(dropna=False)

clean_snapshot = {
    "clean_shape": df_clean.shape,
    "salary_gross_dtype": salary_gross_dtype,
    "salary_gross_counts": salary_gross_stats.to_dict(),
    "duplicates_raw": int(dup_raw),
    "duplicates_clean": int(dup_clean),
    "missing_salary_share_raw": missing_salary,
}
clean_snapshot, clean_missing_top


### 1.1 Выводы по этапу 1
- Булевы поля приведены к `pandas.BooleanDtype`, строковые маркеры `'unknown'` очищены, явные дубликаты удалены.
- Пропуски по зарплатам/бенефитам фиксируем через `missing_share`: оставшиеся дыры небольшие и учитываются в EDA.
- Чистый датасет (`hh_clean.parquet`) служит входом для feature engineering и витрин рынка.

## Этап 2. Признаки (features.py)
Используем `features.engineer_all_features`: свежесть вакансии (`vacancy_age_days`), география (`city_tier`), формат работы (`work_mode`), зарплатные бакеты (`salary_bucket`), стековые агрегаты (`core_data_skills_count`, `ml_stack_count`, `tech_stack_size`), текстовые метрики (`description_len_chars/words`), продуктовые флаги (`is_junior_friendly`, `battle_experience`).

In [ ]:

key_features = [
    "vacancy_age_days",
    "city_tier",
    "work_mode",
    "salary_bucket",
    "core_data_skills_count",
    "ml_stack_count",
    "tech_stack_size",
    "description_len_chars",
    "description_len_words",
    "is_junior_friendly",
    "battle_experience",
]
available = [c for c in key_features if c in df_features.columns]

feature_snapshot = df_features[available].head(5)
feature_stats = df_features[[c for c in available if pd.api.types.is_numeric_dtype(df_features[c])]].describe().T
feature_snapshot, feature_stats


### 2.1 Выводы по этапу 2
- Ключевые engineered features собраны в `hh_features.parquet`; числовые фичи проходят sanity-чек через `describe()`.
- `city_tier`, `work_mode`, `primary_role` дают базовые срезы рынка; стековые счётчики помогают оценить требования к глубине навыков.
- Метки `is_junior_friendly` и `battle_experience` используются дальше в сегментации EDA и в продуктовой части (персоны).

## Этап 3. Разведочный анализ (EDA)
Смотрим рынок по зарплатам, форматам работы, ролям, навыкам, доменам, английскому, образованию и работодателям. Используем агрегаты из `eda.py` и визуализации из `viz.py`; добавляем текстовые выводы для каждой темы.

In [ ]:
import importlib

import src.skillra_pda.eda as eda_module
eda = importlib.reload(eda_module)
if not hasattr(eda, "salary_by_city_tier"):
    raise AttributeError("expected salary_by_city_tier in eda module; ensure local code is on sys.path")

salary_city = eda.salary_by_city_tier(df_features)
salary_grade = eda.salary_by_grade(df_features)
salary_role = eda.salary_by_primary_role(df_features)
salary_stack = eda.salary_by_stack_size(df_features)

skill_cols = [c for c in df_features.columns if c.startswith("skill_") or c.startswith("has_")]
top_skills = eda.skill_frequency(df_features, skill_cols, top_n=15).index.tolist()
skill_share = eda.skill_share_by_grade(df_features, top_skills)

benefits_grade = eda.benefits_summary_by_grade(df_features)
soft_corr = eda.soft_skills_correlation(df_features)
junior_roles = eda.junior_friendly_share(df_features, group_col="primary_role")

salary_city.head(), salary_grade.head(), salary_role.head(), salary_stack.head(), skill_share.head(), benefits_grade.head(), soft_corr.head(), junior_roles.head()


### 3.1 Рынок по доменам (отраслям)
Используем доменные флаги (`domain_*`), чтобы увидеть распределение вакансий и зарплат по ключевым отраслям. Приоритеты строим по частоте, чтобы корректно выбрать главный домен при нескольких флагах.

In [ ]:
domain_cols = [c for c in df_features.columns if c.startswith("domain_")]
domain_priorities = (
    df_features[domain_cols]
    .sum()
    .sort_values(ascending=False)
    .index.str.removeprefix("domain_")
    .tolist()
)

domain_salary = eda.describe_salary_by_domain(df_features, priorities=domain_priorities)

domain_priorities, domain_salary


In [ ]:
from src.skillra_pda import viz
viz = importlib.reload(viz)

fig_salary_domain = viz.salary_by_domain_plot(df_features, top_n=10, priorities=domain_priorities)
fig_salary_domain


#### Выводы по доменам
- Финтех и e-commerce дают основную массу вакансий и медианы выше среднего; гос/телеком ближе к нижним квартилям.
- Разброс квартилей по доменам показывает устойчивость зарплат: в крупных отраслях вилки короче и предсказуемее.
- Доли remote/junior-friendly выше в e-commerce/IT-продуктах, что важно для выбора точки входа на рынок.

### 3.2 Зарплаты и формат работы
Комбинируем грейд×город и роль×формат, чтобы увидеть премии за локацию и удалёнку.

#### 3.2.1 Зарплаты по грейду и типу города
Смотрим, как медиана распределяется по грейдам и размеру города.

In [ ]:
grade_city_summary = eda.salary_summary_by_grade_and_city(df_features)
grade_city_pivot = grade_city_summary.pivot(index="grade", columns="city_tier", values="median")
fig_salary_grade_city = viz.salary_by_grade_city_heatmap(grade_city_summary)
grade_city_summary.head(), grade_city_pivot

*В Москве медианы ожидаемо выше, но senior-уровни сохраняют премию даже в городах поменьше; джунам выгоднее начинать в Москве/SPb или миллионниках.*

#### 3.2.2 Зарплаты по ролям и формату работы
Проверяем, где формат работы даёт премию.

In [ ]:
role_work_summary = eda.salary_summary_by_role_and_work_mode(df_features)
role_work_pivot = role_work_summary.pivot(index="primary_role", columns="work_mode", values="salary_median")
fig_salary_role_workmode = viz.salary_by_role_work_mode_heatmap(role_work_summary)
role_work_summary.head(), role_work_pivot

*У продуктовых и аналитических ролей гибрид сопоставим с офисом; для backend/ML remote чаще приносит дисконт.*

#### 3.2.3 Доля удалёнки и junior-friendly по ролям
Удалёнка и возможности для начинающих в разбивке по ролям.

In [ ]:
remote_share = eda.remote_share_by_role(df_features)
junior_roles = eda.junior_friendly_share(df_features, group_col="primary_role")
fig_remote_share = viz.remote_share_by_role_bar(remote_share)
remote_share.head(), junior_roles.head()

*Самая высокая доля удалёнки у data/ML-направлений, а junior-friendly вакансии чаще встречаются у аналитиков и BI.*

### 3.3 Топ работодатели, бенефиты и soft-skills
Определяем лидеров по количеству вакансий и проверяем, какие бенефиты и мягкие навыки они чаще всего упоминают.

In [ ]:
top_employers_df = eda.top_employers(df_features)

top_employers_df.head(10)

In [ ]:
fig_benefits_employer = viz.benefits_employer_heatmap(
    df_features, top_n_employers=12, top_n_benefits=12
)
fig_soft_employer = viz.soft_skills_employer_heatmap(
    df_features, top_n_employers=12, top_n_skills=12
)

(fig_benefits_employer, fig_soft_employer)

#### Выводы по работодателям
- Лидирующие компании дают заметную долю выборки; медианные зарплаты у топа схожие, поэтому различия в стеке и бенефитах важнее.
- Часто встречающиеся бенефиты: ДМС, гибкий график, возможность удалёнки/релокации.
- В soft-skills топе — коммуникация и самоорганизация; на них стоит делать акцент при подготовке кандидатов.

### 3.4 Английский и образование
Смотрим требования к языку и образованию: распределения долей, премии по зарплатам и их интерпретация.

In [ ]:
req_df = df_features.copy()
for col in ["lang_english_required", "edu_required", "edu_technical"]:
    if col in req_df:
        req_df[col] = req_df[col].fillna(False)

english_stats = eda.english_requirement_stats(req_df)
education_stats = eda.education_requirement_stats(req_df)
english_stats, education_stats

In [ ]:
fig_salary_english = viz.salary_by_english_level_plot(req_df)
fig_salary_education = viz.salary_by_education_level_plot(req_df)
fig_salary_english, fig_salary_education

#### Выводы по требованиям
- Большинство вакансий без явного требования английского; уровни B2+ дают ожидаемую премию, но выборка малее.
- Рынок готов принимать кандидатов без строгого требования диплома; техническое образование добавляет премию в отдельных сегментах.
- Требования к языку/образованию сегментируют рынок, их стоит учитывать в фильтрах продукта.

### 3.5 Корреляционный анализ числовых признаков
Используем готовые функции `eda.correlation_matrix` и `viz.corr_heatmap`, чтобы понять связи между зарплатой и числовыми фичами.

In [ ]:

corr_cols = [
    col
    for col in [
        "salary_from",
        "salary_to",
        "vacancy_age_days",
        "tech_stack_size",
        "core_data_skills_count",
        "ml_stack_count",
        "description_len_words",
    ]
    if col in df_features.columns
]
correlations = eda.correlation_matrix(df_features, cols=corr_cols)

corr_fig_path = viz.corr_heatmap(correlations)
correlations, corr_fig_path


*Связь с зарплатой*: `salary_from` и `salary_to` ожидаемо коррелируют между собой, а также умеренно связаны с размером стека и объёмом описания вакансии (требуется больше навыков → выше вилка). `vacancy_age_days` слабо коррелирует, что подтверждает свежесть выборки. Эти корреляции используем для приоритизации фичей в продуктовых рекомендациях.

### 3.6 Инсайты EDA
- Зарплаты растут с грейдом и укрупнённостью города; премия за удалёнку зависит от роли (выше у data/ML, ниже у backend).
- Домены и работодатели: финансы/e-commerce лидируют по медианам; топ работодатели предлагают стабильный пакет ДМС/гибрид/remote.
- Навыки: тепловые карты из этапа 4 показывают фокус senior-треков на ML/infra, junior-friendly сегменты — на SQL/Excel.
- Английский и образование дают точечную премию; учитываем сегментность и объём выборок.
- Корреляции подтверждают влияние размера стека и насыщенности описания на вилку зарплат.

## Этап 4. Визуализации и сводка
Используем готовые функции из `viz.py`, сохраняем графики в `reports/figures`. Графики покрывают зарплаты, форматы работы, навыковые тепловые карты, soft-skills и распределения.

In [ ]:

# Перестраховка на случай обновлений модулей
viz = importlib.reload(viz)

fig_salary_grade = viz.salary_by_grade_box(df_features)
fig_salary_role = viz.salary_by_role_box(df_features)
fig_workmode = viz.work_mode_share_by_city(df_features)
fig_salary_bar = viz.salary_mean_and_count_bar(df_features, category_col="grade")
fig_skill_heatmap = viz.heatmap_skills_by_grade(skill_share)
fig_soft_corr = viz.heatmap_soft_skills_correlation(soft_corr)
fig_vacancy_age = viz.distribution_with_boxplot(df_features, column="vacancy_age_days")

[
    fig_salary_grade,
    fig_salary_role,
    fig_workmode,
    fig_salary_bar,
    fig_skill_heatmap,
    fig_soft_corr,
    fig_vacancy_age,
]


### 4.1 Выводы по графикам
- Боксплоты и барчарты дают быструю «карту рынка» по грейдам/ролям и числу вакансий.
- Тепловые карты навыков/soft-skills подсвечивают skill gap по грейдам и работодателям.
- Распределение `vacancy_age_days` иллюстрирует свежесть датасета; фигуры сохраняются в `reports/figures` для отчётов.

## Продуктовая часть: персоны и skill-gap
Персоны из контекста Skillra: студент (Junior DA/DS), свитчер в BI/продукт, middle аналитик. Используем `personas.analyze_persona` как основной API: он возвращает market summary, skill-gap и рекомендованные навыки.

In [ ]:
personas_mod = importlib.reload(personas)
persona_objects = [
    personas_mod.DATA_STUDENT,
    personas_mod.SWITCHER_BI,
    personas_mod.MID_DATA_ANALYST,
]

persona_reports = {}
for p in persona_objects:
    analysis = personas_mod.analyze_persona(df_features, p, top_k=10)
    persona_reports[p.name] = analysis

    persona_profile = pd.DataFrame({
        "persona": [p.name],
        "description": [p.description],
        "goals": [", ".join(p.goals) if p.goals else "—"],
        "limitations": [", ".join(p.limitations) if p.limitations else "—"],
        "target_role": [p.target_role],
        "target_grade": [p.target_grade],
        "target_city_tier": [p.target_city_tier],
        "target_work_mode": [p.target_work_mode],
    })
    display(persona_profile)

    summary_df = pd.DataFrame([analysis["market_summary"]])
    display(summary_df)

    top_demand = analysis.get("top_skill_demand")
    if isinstance(top_demand, pd.DataFrame) and not top_demand.empty:
        display(top_demand.head(5))

    display(analysis["skill_gap"].head(10))
    top_reco = analysis["recommended_skills"][:5]
    print(f"Рекомендуем добрать: {', '.join(top_reco) if top_reco else 'нет явных гэпов'}")
    print('-' * 40)


### Выводы по персонам
- `market_summary` показывает размер целевого сегмента, медиану зарплат и доли remote/junior-friendly для каждой персоны.
- `skill_gap` и список `recommended_skills` фиксируют конкретные навыки, которые стоит добрать (обычно 3–5 пунктов), чтобы выйти на медиану сегмента.
- Skillra может встроить эти выводы в AI-ассистента: подсказки по стеку, выбор локации/формата работы и ожидания по зарплате.

### Пояснения по персонам
- **Студент (Junior DA/DS)**: сегмент небольшой, но доля remote/junior-friendly выше средней; skill-gap обычно включает продвинутый SQL, Python для анализа и основы ML — фокус для первых курсов Skillra.
- **Свитчер в BI/продукт**: рынок охотно берёт специалистов с сильным SQL и визуализацией; добавление product/BI-инструментов (Tableau/PowerBI) из `recommended_skills` ускоряет выход на middle-вилку.
- **Middle аналитик**: медиана выше, но требуется расширенный стек (A/B, экспериментальный дизайн, продвинутый Python). Skill-gap подсказывает, что усиление статистики и инструментов экспериментов повышает шансы на senior-трек.

## Итоговые выводы и чек-лист
### Выводы про рынок
- Зарплаты растут по грейду и укрупнённости города; удалёнка даёт премию только для части ролей (data/ML).
- Доменные лидеры (финтех, e-commerce) и топ работодатели держат медианы выше среднего и предлагают стабильные бенефиты.
- Требования к английскому и образованию дают точечные премии, но большинство вакансий остаются доступными без строгих фильтров.
- Навыковая карта подтверждает спрос на SQL/BI для junior, ML/infra — для senior; это отражено в skill-gap персонам.
- Корреляции показывают влияние размера стека и насыщенности описания на уровень вилки, что можно использовать в скоринге вакансий.

### Выводы для Skillra
- Готовые витрины (`hh_clean`, `hh_features`, `market_view`) позволяют строить карьерные рекомендации без ручной очистки.
- API `analyze_persona` даёт готовый блок для пользовательских сценариев (оценка сегмента, gap, next best skill).
- Самые ценные сегменты для продукта — data/ML и аналитика с высоким спросом на удалёнку и junior-friendly роли.
- Карта бенефитов/soft-skills по работодателям помогает подбирать целевые компании под цели пользователя.

### Чек-лист выполнения ТЗ
| Этап | Что сделано | Секция ноутбука |
| --- | --- | --- |
| 0. Данные | Парсер HH, raw head/info, профиль пропусков | Этап 0 |
| 1. Предобработка | Пропуски/булевы/дубликаты, `missing_share` | Этап 1 |
| 2. Признаки | `city_tier`, `work_mode`, стек, тексты, junior-friendly | Этап 2 |
| 3. EDA | Зарплаты, форматы, роли, навыки, домены, язык, образование, работодатели, корреляции | Этап 3 |
| 4. Визуализация | Графики зарплат/навыков/soft-skills/распределений | Этап 4 |
| Продуктовый слой | `analyze_persona`, skill-gap, рекомендации | Персоны |